# Milestone 1 - Group 21

https://github.com/UBC-MDS/DSCI_525_group21

In [1]:
# load libraries
import io
import os
import json
import glob
#import intake
import requests
import numpy as np
import pandas as pd
#import xarray as xr
from urllib.request import urlretrieve
#import proplot as pplot
#from joblib import Parallel, delayed
#import warnings
#warnings.filterwarnings("ignore")  # ignore some annoying matplotlib warnings
from memory_profiler import memory_usage
import zipfile

In [2]:
# more library loading
%load_ext rpy2.ipython
%load_ext memory_profiler

### 3. Downloading the data

In [3]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare/"

In [4]:
# metadata output
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # 
files = data["files"] # we only want the data and readme 'name' key value

In [5]:
%%time
#download readme and data.zip files only
files_to_dl = ["README.md", "data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 5.5 s, sys: 4.64 s, total: 10.1 s
Wall time: 3min 14s


In [6]:
%%time
#extract zip files to repo
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 16.1 s, sys: 2.75 s, total: 18.8 s
Wall time: 19.7 s


### 4. Combining data CSVs

In [7]:
%%time
%memit

# Shows time that regular python takes to merge file
# Join all data together
import pandas as pd
use_cols = ["time",'lat_min','lat_max','lon_min', 'lon_max', 'rain (mm/day)']

files = glob.glob('./figshare/*.csv')
df_all = None

for file in files:
    filename = os.path.basename(file)
    
    if '_daily_rainfall_NSW.csv' in filename:
        print(f"Processing the file {filename}")
        model = filename.split('_daily_rainfall_NSW.csv')[0]

        df = pd.read_csv(file, usecols=use_cols, index_col=0)
        df['model'] = model    

        if df_all is None:
            df_all = df
        else:
            df_all = df_all.append(df)

peak memory: 155.11 MiB, increment: 0.21 MiB
Processing the file AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
Processing the file MIROC6_daily_rainfall_NSW.csv
Processing the file FGOALS-f3-L_daily_rainfall_NSW.csv
Processing the file NorESM2-MM_daily_rainfall_NSW.csv
Processing the file EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
Processing the file CanESM5_daily_rainfall_NSW.csv
Processing the file NESM3_daily_rainfall_NSW.csv
Processing the file GFDL-ESM4_daily_rainfall_NSW.csv
Processing the file MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
Processing the file ACCESS-CM2_daily_rainfall_NSW.csv
Processing the file FGOALS-g3_daily_rainfall_NSW.csv
Processing the file INM-CM5-0_daily_rainfall_NSW.csv
Processing the file MRI-ESM2-0_daily_rainfall_NSW.csv
Processing the file MPI-ESM1-2-HR_daily_rainfall_NSW.csv
Processing the file SAM0-UNICON_daily_rainfall_NSW.csv
Processing the file NorESM2-LM_daily_rainfall_NSW.csv
Processing the file BCC-CSM2-MR_daily_rainfall_NSW.csv
Processing the file TaiESM1_dai

In [8]:
# save combined file
df_all.to_csv('./figshare/combined_data.csv')

In [9]:
%%sh 
#get file size of combined csv
du -sh figshare/combined_data.csv

5.6G	figshare/combined_data.csv


**Observations**

Our team members had the following computer specs:  

| Team Member       | Ram     | Processor     |
| :------------- | :----------: | -----------: |
|  Cal | 16GB   | AMD Ryzen 5 3600 6-core    |
| Justin  | 32GB  | Intel i5 | 
| Anita   |  12GB | Intel i5  | 
| Yuan  |  8GB | Intel i5  | 

Below are our processing times and peak memory usage by team member: 

| Team Member       |  Processing Time     | Peak Memory Usage |
| :------------- | :---------- | :-----------: |
|  Cal |  1min 13sec  | 137 mb |
|  Justin |  1min 24sec  | 155 mb |
|  Anita |  3min 34sec  | 9050 mb |
|  Yuan |  9sec (using Dash)  | 250 mb |

We also used the Pandas default writing method.

### 5. Load the combined CSV to memory and perform a simple EDA

**Performance using Default Pandas Method**

In [10]:
%%time
%%memit
# time to read/calculate when using default Pandas method
df = pd.read_csv("figshare/combined_data.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
TaiESM1             3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NESM3                966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
Name: model, dtype: int64
peak memory: 12154.78 MiB, increment: 8446.80 MiB
CPU times: user 51.3 s, sys: 3.97 s, total: 55.3 s
Wall time: 55.4 s


**Performance when loading in Select Columns only**

In [11]:
%%time
%%memit
use_cols = ["time", "rain (mm/day)", "model"]
df = pd.read_csv("figshare/combined_data.csv", usecols = use_cols)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
TaiESM1             3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NESM3                966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
Name: model, dtype: int64
peak memory: 13288.14 MiB, increment: 4633.28 MiB
CPU times: user 42.7 s, sys: 3.74 s, total: 46.5 s
Wall time: 46.7 s


**Performance when reading file using chunks**

In [12]:
%%time
%%memit
# 10 million chunk size
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshare/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
peak memory: 8988.77 MiB, increment: 2193.34 MiB
CPU times: user 53.2 s, sys: 3.42 s, total: 56.6 s
Wall time: 56.7 s


In [13]:
%%time
%%memit
# 1 million chunk size
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshare/combined_data.csv", chunksize=1_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
peak memory: 7117.98 MiB, increment: 99.35 MiB
CPU times: user 57.2 s, sys: 1.45 s, total: 58.7 s
Wall time: 58.8 s


In [14]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshare/combined_data.csv", chunksize=500_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
peak memory: 6945.82 MiB, increment: 31.89 MiB
CPU times: user 56 s, sys: 1.06 s, total: 57.1 s
Wall time: 57.2 s


**Performance when loading with simplier data types**

In [15]:
df.dtypes

time              object
rain (mm/day)    float64
model             object
dtype: object

In [16]:
%%time
%%memit
col_type = {'time':object, 'lat_min':np.float32, 'lat_max':np.float32, 'lon_min': np.float32, 
           'lon_max': np.float32, 'rain (mm/day)': np.float32, 'model': object}
df2 = pd.read_csv("figshare/combined_data.csv", dtype=col_type)
print(df2["model"].value_counts())

MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
TaiESM1             3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NESM3                966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
Name: model, dtype: int64
peak memory: 12867.46 MiB, increment: 5923.33 MiB
CPU times: user 54.1 s, sys: 3.74 s, total: 57.9 s
Wall time: 58 s


In [17]:
df2.dtypes

time              object
lat_min          float32
lat_max          float32
lon_min          float32
lon_max          float32
rain (mm/day)    float32
model             object
dtype: object

**Using Dask**

In [37]:
%%time
%%memit
import dask.dataframe as dd

# time to read/calculate when using default Dask method
d_df = dd.read_csv("figshare/combined_data.csv")
result = d_df.model.value_counts().compute()

print(result)


MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 20956.89 MiB, increment: 677.56 MiB
CPU times: user 1min 21s, sys: 6.85 s, total: 1min 28s
Wall time: 33.3 s


*observations*

For part 5, we tasked each of our team members to investigate each one of the approaches. Please see the summary of results below which were performed on Justin's machine (i5 processor (4 core, 8 threads), 32GB memory).

* Default Pandas approach
* Changing the Data Type of the columns using float32 instead of float64 for 4 out of 7 columns
* Reading in fewer columns (date, rain and model)
* Reading in using several chunk sizes (0.5M, 1M and 10M chunks)
* Loading with Dask

| Approach Taken       |  Processing Time     | Peak Memory Usage |  Increment Memory Usage |
| :------------- | :---------- | :----------- | :-----------: |
|  Default Pandas | 55s  | 12,155 MB | 8,447 MB |
| Change Data Type | 58s   |  12,867 MB  | 5,923 MB   |
|  Fewer Columns |  47s  | 13,288 MB | 463 MB |
|  Loading in 0.5 M chunk size|  57s  | 8,989 MB | 2,193 MB |
|  Loading in 1 M chunk size|  59s  | 7,118 MB | 99 MB |
|  Loading in 10 M chunk size|  57s  | 6,946 MB | 32 MB |
|  Dask |  1m 28s  | 20,957 MB | 677MB |





The fastest approach appears to be using fewer columns and the slowest approach was Dask. There are probably more advanced tuning in Dask to improve this but we only used the default settings.

Note that:
* **Peak memory**: peak memory usage of your system (including memory usage of other processes) during the program runtime.
* **Increment**: the increment in memory usage relative to the memory usage just before the program is run 

The lowest peak memory usage is using chunking with 10M chunk size and the highest peak memory usage is using the default Dask.

The lowest increment memory usage is using chunking with 10M chunk size and the highest peak memory usage is using the default Pandas approach.



### 6. Perform a simple EDA in R

In [18]:
import pandas as pd
## install the packages https://arrow.apache.org/docs/python/install.html
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
## How to install put instructions https://anaconda.org/conda-forge/rpy2
import rpy2.rinterface
# install this https://pypi.org/project/rpy2-arrow/#description  pip install rpy2-arrow
# have to install this as well conda install -c conda-forge r-arrow 
import rpy2_arrow.pyarrow_rarrow as pyra
### instruction
import pyarrow.feather as feather

In [19]:
%%R
#just seeing if its available
library("arrow")
library("dplyr")

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [20]:
%%time
%%memit
## read more on the datasets here  https://arrow.apache.org/docs/python/dataset.html
dataset = ds.dataset("figshare/combined_data.csv", format="csv")
## this is of arrow table format
table = dataset.to_table()

peak memory: 14884.18 MiB, increment: 4126.20 MiB
CPU times: user 23.5 s, sys: 2.52 s, total: 26 s
Wall time: 24.9 s


In [21]:
%%time
# experiment in writing in feather format 
#feather.write_feather(table, 'figshare/figshare.feather')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


In [22]:
%%time
%%R
### her we are showing how much time it took to read a feather file what we wrote in python
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("figshare/figshare.feather")
print(class(r_table))
library(dplyr)
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

[1] "tbl_df"     "tbl"        "data.frame"
# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 8.571577 secs
CPU times: user 7.56 s, sys: 5.71 s, total: 13.3 s
Wall time: 8.64 s


## Why I chose Feather?

We chose the feather data format - which was designed to improve data interoperability (ex. language agnostic - R and Python) while dealing with columnar tabular data. I also wanted the fastest load and save times. Feather is a fast, lightweight, and easy-to-use binary file format. Feather also doesn't use compression internally and works best with SSD drives.

However, if I needed to store this for long term storage, I would not use Feather because the file format is relatively new and can change. Feather does not guarantee if the format will stay the same between versions.